# House of Pymoli Data Analysis
* Players in the 35-39 age bracket spent the most per person (\\$4.76).  Interestingly, players over \\$40 were the least profitbale demographic, spending, on average, \\$1.57 less than those 35-39 (\\$3.19). <br>
* While females made up less than 15\% percent of the player base, they tended to outspend their male counterparts by \\$0.18 per purchase. <br>
* Game prices ranged from \\$1.00 to \\$4.99, with an average price of \\$3.05.  The most profitabale games were all priced in the top quartile of game prices (\\$4.08+).<br>

In [65]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

## Player Count
#### Determine total player counts

In [61]:
#remove duplicate screen names ("SN")
players_df = df["SN"].unique()

#Determine number of players and store as variable
player_count = len(players_df)

#create & display a player count dataframe
player_count_df = pd.DataFrame({"Total Players": [player_count]})
player_count_df

,Total Players
0,576


## Purchasing Analysis (Total)
#### Determine number of unique items, average price, total purchases & total revenue 

In [62]:
#Find numner of unique items
unique_items = len(df['Item ID'].unique())

#calculate average price, total number of unique purches & sum of all revenue
avg_price = df['Price'].mean()
total_purchases = len(df['Purchase ID'].unique())
total_revenue = df["Price"].sum()

#format output display to 2 decimals
pd.options.display.float_format = '${:,.2f}'.format

#create dataframe with calculations
df_purchases = pd.DataFrame({
    "Number of Unique Items" : [unique_items],
    "Average Price": [avg_price],
    "Number of Purchases": [total_purchases],
    "Total Revenue": [total_revenue]                
})

#display purchasing analysis data
df_purchases

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics
#### Determine gender percentages of player base

In [26]:
#create dataframe of unique players by gender
gender_df = df[["SN","Gender"]]
gender_unique_df = gender_df.drop_duplicates(['SN'])

#group & sort dataframe by gender
gender_df=gender_unique_df.groupby(['Gender'])
gender_sorted_df = gender_df.count().sort_values(['SN'], ascending=False)
gender_sorted_df.reset_index(inplace=True)

#calculate percentages
prctMale = gender_sorted_df.iloc[0,1] / player_count * 100
prctFemale = gender_sorted_df.iloc[1,1] / player_count * 100
prctOther = gender_sorted_df.iloc[2,1] / player_count * 100

#format output to 2 decimal places as percentage
pd.options.display.float_format = '{:,.2f}%'.format

#create sorted dataframe with gender calculations
gender_sorted_df['Percentage of Players'] = pd.DataFrame({'Percentage of Players': [prctMale, prctFemale, prctOther]})
gender_analysis_df = gender_sorted_df.set_index('Gender')

#diplay gender dataframe
gender_analysis_df = gender_analysis_df.rename(columns={'SN': 'Total Count'})
del gender_analysis_df.index.name
gender_analysis_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)
#### Determine purchasing statistics by gender

In [27]:


#create dataframe with gender & price info only
df_genders_all = df[['Gender','Price']]
#group dataframe by gender
df_grouped_gender = df_genders_all.groupby(['Gender'])
#calculate purchases made by each gender & store into new dataframe
df_grouped_gender_counts = df_grouped_gender.count()
df_grouped_gender_analysis = df_grouped_gender_counts.rename(columns={'Price':'Purchase Counts'})

#calculate average & total purchases by gender
price_mean = df_grouped_gender.mean()
total_value = df_grouped_gender.sum()
#store calculations in dataframe
df_grouped_gender_analysis["Average Purchase Price"] = price_mean
df_grouped_gender_analysis["Total Purchase Value"] = total_value
#calculate averages by gender
avg_Total_Female = total_value.iloc[0,0] / gender_analysis_df.iloc[1,0]
avg_Total_Male = total_value.iloc[1,0] / gender_analysis_df.iloc[0,0]
avg_Total_Other = total_value.iloc[2,0] / gender_analysis_df.iloc[2,0]

#create summary dataframe & group and set index
avg_total_df = pd.DataFrame({
    'Avg Total': [avg_Total_Female, avg_Total_Male, avg_Total_Other]
})
avg_total_df = avg_total_df.set_index(df_grouped_gender_analysis.index)

#add avg total purchase per person column to summary dataset
df_grouped_gender_analysis['Average Total Purchase per Person'] = avg_total_df

#reset output formatting to $X.XX & display summary dataset
pd.options.display.float_format = '${:,.2f}'.format
df_grouped_gender_analysis

,Purchase Counts,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics
#### Detemrine percentages of players in each age bracket

In [28]:
#create dataframe with age information only and eliminate duplicate screen names
age_df = df[['SN','Age']]
age_df = age_df.drop_duplicates(['SN'])

#establish age bins and labels
bins = [0,10,15,20,25,30,35,40,100]
labels = ['<10', '10-14','15-19','20-24','25-29','30-34','35-39','40+']

#sort age dataframe into age bins & group by bin
age_df['Age Category'] = pd.cut(age_df['Age'], bins=bins, right=False, labels=labels)
age_df_binned=age_df[['Age', 'Age Category']]
age_df_grouped = age_df_binned.groupby(['Age Category'])

#calcualte number of players 
bin_ages_df = age_df_grouped.count()
bin_ages_df = bin_ages_df.rename(columns={'Age':'Total Counts'})

#calculate perventages of players in each age group and add to summary dataset
bin_ages_df['Percentage of Players'] = bin_ages_df / len(age_df) *100

#reset output display to X.XX% & display dataset
del bin_ages_df.index.name #matches example formatting
pd.options.display.float_format = '{:,.2f}%'.format
bin_ages_df

,Total Counts,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)
#### Determine purchasing statistics by age

In [31]:
#create dataframe with relevant information
purchase_data_df = df[['SN','Age','Purchase ID','Price']]
purchase_data_nodup_df = purchase_data_df.drop_duplicates(['SN']) #nodup --> no duplicates

#establish age bins and labels
bins = [0,10,15,20,25,30,35,40,100]
labels = ['<10', '10-14','15-19','20-24','25-29','30-34','35-39','40+']

#create dataframes (with duplicates and without) and sort into bins
purchase_data_df['Age Category'] = pd.cut(purchase_data_df['Age'], bins=bins, right=False, labels=labels)
purchase_data_nodup_df['Age Category'] = pd.cut(purchase_data_nodup_df['Age'], bins=bins, right=False, labels=labels)

#create dataframes for calculations
purchase_data_df_binned_count=purchase_data_df[['Age Category','Purchase ID']]
purchase_data_df_binned_price=purchase_data_df[['Age Category','Price']]
purchase_data_df_binned_users=purchase_data_nodup_df[['Age Category','Price']]

#sort dataframes by groups and calculate relevant statistics
purchase_count_df = purchase_data_df_binned_count.groupby(['Age Category']).count()
purchase_price_mean_df = purchase_data_df_binned_price.groupby(['Age Category']).mean()
purchase_price_total_df = purchase_data_df_binned_price.groupby(['Age Category']).sum()
purchase_price_count_df = purchase_data_df_binned_users.groupby(['Age Category']).count()
purchase_price_avg_df = purchase_price_total_df / purchase_price_count_df

#create and populate summary dataframe
purchase_df = purchase_count_df
purchase_df['Average Purchase Price'] = purchase_price_mean_df
purchase_df['Total Purchase Price'] = purchase_price_total_df
purchase_df['Avg Total Purchase per Person'] = purchase_price_avg_df
purchase_df = purchase_df.rename(columns={'Purchase ID':'Purchase Count'})

#reset output display to $X.XX & display formatted dataset
del purchase_df.index.name
pd.options.display.float_format = '${:,.2f}'.format
purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders
#### Determine top spenders & spending statistics

In [59]:
#create dataframe with purchase info for all users
df_purchases = df[['SN', 'Purchase ID']]

#group by user name and calcuulate the count of purchases
df_purchases = df_purchases.groupby('SN').count()

#store purchase counts into new dataframe
df_top = df_purchases.sort_values(['Purchase ID'], ascending=False)
df_top.reset_index(inplace=False)
df_purchase_count['Purchase Count'] = df_top['Purchase ID']

#repeat steps above, but store the sum of all purchases
df_purchase_total = df[['SN', 'Price']]
df_purchase_total = df_purchase_total.groupby('SN').sum()
df_purchase_total = df_purchase_total.sort_values(['Price'], ascending=False)
df_purchase_total.reset_index(inplace=False)
df_purchase_total_spent = df_purchase_total['Price']

#concatenate purchase count and total price dataframes
df_summary = pd.concat([df_purchase_count,df_purchase_total_spent], axis=1)

#calculate average price using newly concatenated dataframe
df_summary['Average']= df_summary['Price'] / df_summary['Purchase ID'] 

#sort dataframe
df_summary = df_summary.sort_values(['Price'], ascending=False)
df_summary.reset_index(inplace=False)

#rename columns
df_summary_top5 = df_summary[['Purchase ID','Average','Price']]
df_summary_top5 = df_summary_top5.rename(columns={'Purchase ID':'Purchase Count',
                                                  'Average':'Average Purchase Price',
                                                  'Price':'Total Purchase Value'})
df_summary_top5 = df_summary_top5.rename_axis('SN')

#dispaly top 5 spenders
df_summary_top5.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items
#### Determine post frequently purchases items

In [48]:
#create dataframe with Item info, price
df_item=df[["Item ID",'Item Name','Price']]
#group by Item info, calculate and store counts; sort by count in new dataframe
df_item_count = df_item.groupby(['Item ID', 'Item Name']).count()
df_item_count_sorted = df_item_count.sort_values(['Price'],ascending=False)
df_item_count_sorted.reset_index(inplace=False)
df_item_count_summary = df_item_count_sorted.rename(columns={'Price':'Purchase Count'})

#repeat for sum of prices
df_item_count_price = df_item.groupby(['Item ID', 'Item Name']).sum()
df_item_count_price_sorted = df_item_count_price.sort_values(['Price'], ascending=False)
df_item_count_price_sorted.reset_index(inplace=False)
df_item_count_price_summary = df_item_count_price_sorted.rename(columns={'Price':'Total Purchase Value'})

#concatenate dataframes into new data frame (sort by purchase count)
df_item_summary = pd.concat([df_item_count_summary, df_item_count_price_summary], axis=1)
df_item_summary = df_item_summary.sort_values(['Purchase Count'], ascending=False)
df_item_summary.reset_index(inplace=False)

#determine item price from calculations
df_item_summary['Item Price'] = df_item_summary['Total Purchase Value'] / df_item_summary['Purchase Count']
df_item_summary_top5 = df_item_summary[['Purchase Count', 'Item Price','Total Purchase Value']]

#print summary dataframe
df_item_summary_top5.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items
#### Determine most profitable items sold

In [49]:
#create dataframe from previous section and sort by Total Purchase Value
df_profitable = df_item_summary_top5.sort_values('Total Purchase Value', ascending=False)

#displace summary dataset
df_profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


## Price Statistics

In [64]:
df['Price'].describe()

count   $780.00
mean      $3.05
std       $1.17
min       $1.00
25%       $1.98
50%       $3.15
75%       $4.08
max       $4.99
Name: Price, dtype: float64